# 딥러닝 ③ Focus on TRANSFORMER

# 1. 인트로

## 1-1. RNN의 한계와 어텐션의 등장 (복습)

> **RNN의 한계**

 RNN(Recurrental Neural Network)은 **순차적인 데이터**(문장, 음악, 날씨 등)를 처리하기 위해  이전 단계의 정보들을 다음 단계로 전달할 수 있도록 하는 인공지능 모델  

하지만 대표적인 한계인 **장기 의존성 문제(Long-term dependency)**와 **Vanishing Gradient** 문제가 있었음


### RNN 복습

 **RNN(Recurrental Neural Network)이란?**

> 앞뒤 내용이 서로 연관된 데이터를 다루기 위해, 이전 단계의 정보를 기억하여 현재 입력과 함께 처리하는 신경망


**RNN의 구조**

- $X$: Input layer  
- $h$: Hidden layer  
- $O$: Output layer  
- $W$: 각 전달 과정의 가중치 (Weight)  
- $b$: Bias (활성화 함수의 출발점 조절 → 안정적 학습에 기여)

 **구조 한줄 요약**

> $X_{t-1}$ 입력으로 $O_{t-1}$ 출력이 나오고, 그 시점의 정보 $h_{t-1}$이 다음 시점의 $h_t$로 전달되어 과거 정보가 계속 이어짐 → 이를 "기억"이라 표현

 **가중치 공유**

> 모든 타임 스텝에서 동일한 가중치를 반복 사용
> 즉, $W_{hx}$, $W_{hh}$, $b_{h}$ 등은 모든 스텝에서 동일하게 적용

**장점**
1. 파라미터 효율성
2. 학습의 일반화 가능

###  장기 의존성 문제 & Vanishing Gradient


RNN은 시퀀스가 길어질수록 초반 정보가 후반에 반영되지 않는 문제가 발생

**Vanishing Gradient**:  
- RNN은 BPTT(Backpropagation Through Time)을 통해 학습. 
- 이때, 시퀀스 길이에 따라 가중치가 반복 곱해지면서 gradient가 점점 0에 수렴하거나 폭주. 

문장이 길수록,  
- 값이 발산하면 → **Exploding Gradient**  
- 값이 0으로 수렴하면 → **Vanishing Gradient**

결과적으로 초반 데이터가 후반에 반영되지 않게 됨



###  LSTM 복습

**LSTM(Long Short-Term Memory) 모델과 한계**

RNN의 Vanishing Gradient 문제를 해결하기 위해 등장.  
핵심 아이디어:  
> "중요한 것은 오래 기억하고, 필요 없는 것은 잊자!"


**구조 개요**
- **Cell State**: 정보가 가중치 곱 없이 직선적으로 전달되는 "메모장"
- **Gate** (정보 선택/관리)
  - Input gate: 새 정보 중 중요한 부분만 저장
  - Forget gate: 불필요한 과거 정보 삭제
  - Output gate: 최종 출력 시 필요한 정보만 사용

 **작동 방식 요약**
> 중요한 정보는 Input gate로 들어가고, 불필요한 정보는 Forget gate로 제거,  
> 필요한 정보만 Output gate로 출력

**장점**
- 장기 기억 가능 (gradient 유지)
- 선택적 정보 저장/삭제

**단점**
- 게이트 3개로 계산 복잡, 속도 느림  
- 여전히 긴 시퀀스에서는 Vanishing Gradient 발생

이유: 초반 데이터는 **더 많은 Forget gate를 통과**해야 하므로 영향이 약해짐. 


##  Attention의 등장

> **Seq2Seq 복습**

Seq2Seq는 **input 시퀀스를 output 시퀀스로 변환**하는 구조입니다.


- 입력/출력 단어 개수가 달라도 가능 (예: 번역)
- **Encoder**: 입력 정보를 읽고 Context Vector로 요약  
- **Decoder**: Context Vector를 바탕으로 출력 생성

예시  
- Encoder → 책을 읽는 사람  
- Context vector → 요약 메모  
- Decoder → 메모를 보고 다른 언어로 다시 쓰는 사람


**Attention의 등장 배경: Seq2Seq의 병목현상**

기존 Seq2Seq는 고정된 크기의 Context Vector로 모든 입력을 요약
→ 문장이 길면 정보 압축이 어려워지고 *입력 정보 손실 발생

해결: **Attention**  
> Decoder가 단어를 생성할 때마다 입력 문장 전체를 다시 보고,  
> 그 시점에서 가장 관련 있는 정보에 집중해 새로운 Context Vector를 만든다!

###  Attention


**핵심 아이디어**  
> 결과를 만들 때, 모든 입력을 동일하게 보지 말고,  
> 현재 시점에서 중요한 부분에 높은 가중치를 부여하자!

**Seq2Seq와의 차이점**
> 출력 시점마다 **새로운 Context Vector를 동적으로 계산**한다 → 병렬화 가능


### Attention 과정

1. **입력 벡터화**
   - 단어들을 수치 벡터로 변환
2. **중요도 점수 산출**
   - 현재 단어와 관련된 입력 단어를 찾음  
   - Score → Softmax → **Attention Weight(가중치)**
3. **중요도 기반 정보 조합**
   - 높은 가중치 단어의 정보를 더 집중적으로 반영

**핵심 프로세스 요약**
1. 입력 시퀀스를 벡터 임베딩으로 변환  
2. 유사도/상관관계 기반 점수 계산 → Softmax 정규화  
3. 가중치를 통해 중요도 반영 (집중 or 무시)


### Attention의 장점

- **시간 유연성 (vs RNN)**  
  → 모든 단어 간 직접 연결, 장기 의존성 해결
- **공간 유연성 (vs CNN)**  
  → 전역적 관계 학습 가능
- **병렬화 (vs RNN)**  
  → Self-Attention을 통해 문장 내 단어 관계를 **동시에 계산**

> 오늘날 Attention은 NLP뿐만 아니라 이미지 생성, 음성 처리, 비전 모델 등 다양한 분야에 사용



### 주요 내용 요약

1. **배경**
   - RNN의 느린 학습, 장기 의존성 문제 → 순차 계산 제거  
   - Encoder~Decoder 전부 **Attention만 사용**
2. **Transformer의 등장**
   - RNN 순환 제거, **Self-Attention 기반 Transformer** 제안
3. **핵심 발견**
   - Recurrence/Convolution 없이도 우수한 성능 달성  
   - **병렬화로 학습 속도 향상**, 대규모 데이터 처리 가능
4. **파급효과**
   - LLM의 탄생 (BERT, GPT 등)
   - NLP 표준화 (번역, 요약, 챗봇 등)
   - 비전/음성/의료 등 다양한 분야 확장



### Transformer란?


> **RNN의 순차 계산 없이, 오직 Attention으로 문장 의미를 파악하는 모델**

기존 Seq2Seq는 **RNN+Attention**,  
Transformer는 **Attention만으로 Encoder/Decoder 구성**

**핵심 기술**
1. **Self-Attention** – 문장 내 단어 간 관계 파악  
2. **Multi-Head Attention** – 여러 관점에서 병렬적 Self-Attention  
3. **Positional Encoding** – 단어 순서 정보 추가

 **즉,**  
기존 RNN 기반 구조를 완전히 대체하고,  
인코더-디코더 모두 Attention 메커니즘으로 구현한 것이 Transformer



# 2. Self-Attention

## 2-1. Query, Key, Value


### Q, K, V 개념

모델이 수많은 정보 안에서 **가장 관련 있는 정보를 효과적으로 찾아내기 (Attention)** 위해  
정보를 세 가지 역할로 분할



###  Q : Query (질문, 요청)

- 지금 내가 알고 싶거나, 초점을 맞추고 있는 대상
- 예시  
  - “서강대학교 인사이트 14기 인원”을 확인하기 위해 검색창에 입력하는 문장 → **Query**
  - Attention에서 문장 속 한 단어가 다른 단어들과의 관계를 파악할 때, **그 기준이 되는 단어 자신**이 Query



###  K : Key (이름표, 색인)

- 검색 대상이 되는 모든 정보들이 달고 있는 **‘이름표’**
- Query는 이 Key들과 자신을 비교하며 얼마나 관련 있는지를 확인
- 예시  
  - “서강대 학생 데이터베이스의 학번” = Key  
  - 문장 내 모든 단어가 Key 역할을 하며, 현재 Query 단어와의 관련도를 평가



### V : Value (실제 내용물)

- Key와 한 쌍으로 묶여 있는 **실제 정보**
- Query–Key 유사도 계산 후, 가장 관련 높은 Key에 해당하는 Value를 가져옴
- 예시  
  - Key(학번)과 관련 높은 학생의 정보(나이, 학과 등)가 Value  
  - Attention에서 Query가 관련 높은 단어(Key)의 의미 정보(Value)를 가져와 자신의 의미를 보강

>  **Key-Value는 쌍으로 묶여 있으며, Key는 Value를 대표하는 역할**



## 2-2. 기존 Cross-Attention vs. Self-Attention

| 구분 | 기존 어텐션 (Cross-Attention) | 셀프 어텐션 (Self-Attention) |
| --- | --- | --- |
| 핵심 아이디어 | 번역할 문장을 보면서 단어를 생성 | 문장 내부 관계를 파악 |
| 작동 방식 | 한 시퀀스가 다른 시퀀스를 참고 | 동일한 시퀀스 내부 단어 관계 계산 |
| 참조 대상 | 서로 다른 시퀀스 (인코더 ↔ 디코더) | 동일 시퀀스 (문장 내부 ↔ 내부) |
| 주요 목적 | 인코더-디코더 연결, 정보 병목 해소 | 문맥 및 의존 관계 파악 |
| 기반 아키텍처 | RNN (LSTM, GRU) | Transformer |
| 병렬 처리 | 불가능 (순차적) | 가능 |


###  기존 Attention (Cross-Attention)

- **Query와 Key-Value 쌍이 서로 다른 시퀀스에서 생성**
- 즉, **Decoder가 Encoder를 참고하는 구조**

#### 예시

입력(Encoder): “The cat is sleeping.”  
출력(Decoder): “고양이가 자고 있다.”

1. **Query 만들기**  
   - Decoder는 지금까지 생성된 단어(“고양이가”)를 보고 **Query 벡터** 생성  
   - 즉, “다음 단어를 만들기 위해 어떤 영어 단어를 참고해야 하지?”

2. **Key, Value 불러오기**  
   - Encoder는 입력 문장을 Key, Value 쌍으로 보관  
     ```
     "The" → K₁, V₁  
     "cat" → K₂, V₂  
     "is" → K₃, V₃  
     "sleeping" → K₄, V₄
     ```

3. **유사도 계산 (Attention Score)**  
   - Q와 각 K의 내적(Q·Kᵢ) → 관련도 계산  
   - “자고” ↔ “sleeping” → 높은 유사도  
   - Softmax로 정규화해 가중치(α) 계산

4. **Weighted Sum**  
   - 가중치로 Value들을 섞어 context vector 생성

5. **Decoder가 다음 단어 예측**  
   - context vector + 자신의 상태로 “자고” 예측

>  **비유:** 오픈북 시험처럼, 디코더가 교과서(입력)를 보며 답안을 작성하는 과정


### Self-Attention

- **문장 자체를 뜯어보며 내부 관계를 파악**
- Query와 Key-Value 쌍이 **같은 시퀀스에서 생성**

> “문장 스스로가 문맥을 이해하는 과정”

####  핵심 과정

1. **단어의 벡터 임베딩 생성**
2. **모든 단어 간 관계 점수 계산 (내적)**
3. **Softmax로 중요도(Attention Weight) 계산**
4. **가중치를 바탕으로 문맥이 풍부해진 벡터 생성**

#### 예시  
> “On Friday, the judge issued a sentence.”

- ‘sentence’는 ‘judge’, ‘issued’와 높은 관련도 → “법적 처벌” 의미로 해석

#### 상세 작동 단계
1. 입력 시퀀스를 임베딩으로 변환  
2. Q·K 내적으로 유사도 계산  
3. Softmax로 0~1 사이의 확률 분포 생성  
4. Value에 가중치 곱 → 문맥 반영된 벡터 출력  

학습 과정에서 **역전파와 경사 하강법**을 통해  
모델이 최적의 벡터 임베딩과 Attention Weight를 학습




## 2-3. Scaled Dot-Product Attention

###  Dot-Product Attention

쿼리(Q)와 키(K) 벡터를 **내적(dot product)** 하여 유사도를 계산하는 방식

1. Q와 K 내적 → Attention Score = Q·K  
2. Softmax 적용 → Attention Weight 계산  
3. 가중치를 Value(V)에 곱해 최종 결과 계산

**문제점**  
- 벡터 차원 $d_k$가 커질수록 내적 결과가 커져  
  Softmax 출력이 왜곡되고, **Vanishing Gradient** 발생



### Scaled Dot-Product Attention

> “Attention is All You Need”에서 제안된 개선된 방식 (Transformer의 기본 구조)

**해결 방법:**  
내적 결과를 $\sqrt{d_k}$ 로 나누어 **스케일링(Scaling)**

####  계산 과정

1. $QK^T$ 계산  
2. $\sqrt{d_k}$ 로 나누어 스케일 조정  
3. Softmax 적용 → Attention Weight 산출  
4. 가중치 × Value(V) → 최종 출력 계산

#### 공식  
$Attention(Q,K,V) = softmax(\frac{QK^T}{\sqrt{d_k}})V$

>  결과 해석  
> - Cross-Attention: 입력 문장에서 어떤 단어에 집중할지 반영한 **Context Vector**  
> - Self-Attention: 문장 내 단어 관계를 반영한 **문맥 표현(Contextual Representation)**




# 3. Multi-Head Attention


###  Multi-Head Attention의 위치


- 인코더(왼쪽 박스 내부)에서는 Multi-Head Attention이 **한 번**,  
  디코더(오른쪽 박스 내부)에서는 **두 번** 일어남
- 특히 디코더의 첫 번째 Multi-Head Attention은 **Masked Multi-Head Attention**이라고 불림
- 즉, Multi-Head Attention은 인코딩된 토큰들을 모델이 “이해할 때” 핵심적으로 사용됨


## 3-1. Why Multi-Head Attention?


###  Single-Head Attention

- 문장 내 단어 간 관계를 파악하기 위해 **하나의 가중치 행렬** $(W^Q, W^K, W^V)$ 만을 학습
- **작동 방식**
  - 512차원 입력 벡터 → 어텐션 가중치 분포 계산 → 가중합으로 512차원 출력 생성
- **한계점**
  - 하나의 어텐션 분포에 문법, 의미, 위치 등 모든 정보를 담다 보니  
    중요한 관계가 **평균화되어 희미해질 수 있음**

**예시**  
> "The animal didn't cross the street because **it** was too tired."

- ‘it’은 **‘animal’**(의미적 연결)과 **‘street’**(문맥적 관계)을 동시에 고려해야 합니다.  
- 하지만 하나의 Head에서는  
  - ‘animal’ → 0.6  
  - ‘street’ → 0.2  
  - 기타 단어 → 0.2  
  로 가중치를 주면, 핵심적인 관계가 다른 정보와 **평균화되어 약화**됨



###  Multi-Head Attention

- Multi-Head Attention은 단어 간 관계를 여러 관점으로 나누어 **병렬로 학습** 
- *Attention is All You Need* 논문에서는  
  512차원 입력 벡터를 **64차원씩 8개(head)** 로 나누어 **병렬 Attention 연산**을 수행


## 3-2. Multi-Head Attention 작동 방식



“I love you” 문장에서 **“love”**라는 단어 벡터가 Multi-Head Attention Layer를 통과하는 과정을 살펴보겠습니다.

가정:
- 입력 벡터 차원 = 512
- Head 개수 = 8
- 각 Head의 차원 = 64


### 1. 분할 (Projection) - 8명의 헤드에 단어 벡터 할당
### 2. 병렬 어텐션 계산 - 각 헤드들이 각자 분석 수
### 3. 결합 및 최종 투영 - 전문가들의 분석 결과 취합






# 4. Transformer 전체 아키텍처


## 4-1. 트랜스포머의 전처리 단계

###  토큰화 (Tokenization)


텍스트 입력을 모델이 이해할 수 있는 **단위(tokens)**로 나누는 과정

- **단어 단위 토큰화:** `Write`, `A`, `Story`
- **문장부호 분리:** `.`, `!`
- **단어 분리:** `doesn’t` → `does`, `n’t`


###  임베딩 (Embedding)



토큰화된 단어를 **숫자 벡터로 변환**하는 단계
즉, **인간의 언어를 컴퓨터의 언어로 번역**하는 과정

- 예: 임베딩 벡터의 차원 수 $d_{model} = 1024$
- **유사한 의미의 단어**는 **가까운 좌표**로 표현
  - 예: `사과(5,5)`는 `딸기(5,4)`와 가깝고 `성(2,1)`과는 멈

> 임베딩은 어텐션이 작동하기 전, 단어 간 의미적 관계를 포착하기 위한 기반을 제공

#### Word2Vec 참고

1. 신경망을 학습시켜 문맥 기반으로 다음 단어를 예측
2. 마지막에서 두 번째 층의 출력을 단어 임베딩으로 사용  
3. 이 층은 단어의 **깊은 속성(deep properties)**을 포착



### Positional Encoding



트랜스포머는 **모든 단어를 동시에 처리**하기 때문에, 단어의 **순서 정보**가 필요
이를 위해 **Positional Encoding**을 통해 순서 정보를 벡터에 추가

> “개는 고양이를 쫓는다” ≠ “고양이는 개를 쫓는다”

####  일관된 규칙(Consistent Sequence)
- 각 위치마다 **항상 동일한 벡터**를 더해줌 
  - 1번째 단어 → A 벡터  
  - 2번째 단어 → B 벡터  
  - 3번째 단어 → C 벡터  

####  수식으로 표현
$$
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d_{model}}}\right)
$$
$$
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d_{model}}}\right)
$$

- 짝수 차원 → $\sin$, 홀수 차원 → $\cos$
- 이렇게 하면 긴 문장에서도 **순서 구분이 명확**해짐



## 4-2. 트랜스포머의 Encoder와 Decoder

###  Encoder


 입력 문장을 이해하고 **요약된 의미 벡터(Context Vector)**로 변환

| 구분 | RNN/LSTM | Transformer |
|------|-----------|--------------|
| 처리 방식 | 순차 입력 | 병렬 입력 |
| 정보 전달 | 이전 은닉상태 전달 | Self-Attention으로 모든 단어 관계 계산 |
| 순서 인식 | 시점 t 내재 | Positional Encoding 사용 |
| 구조 | 반복 셀 | Multi-Head Attention + FFN 반복 |
| Context Vector | 마지막 은닉값 1개 | 모든 단어별 Context |



####  Encoder 구조

1. **Multi-Head Attention**
2. **Feed Forward Network (FFN)**
3. **Residual Connection + Layer Normalization**

#####  Feed Forward Layer (FFN)
- 입력 벡터를 확장하고 비선형 함수(ReLU) 적용  
- Attention이 "무엇을 볼지" 정했다면, FFN은 "깊게 분석"함

#####  Residual Connection
- 입력을 출력에 더해줌 → 기울기 소실 문제 완화  
- ResNet 아이디어에서 차용

#####  Layer Normalization
- 입력의 평균과 분산으로 정규화  
- 학습 안정화 및 빠른 수렴 유도


### Decoder



 **Encoder의 출력을 받아** 한 단어씩 **출력 문장을 생성**

| 비교 항목 | Encoder | Decoder |
|------------|----------|----------|
| 정보 방향 | 양방향 | 단방향 (왼→오) |
| 처리 방식 | 병렬 | 학습은 병렬, 예측은 순차 |
| 역할 | 입력 이해 | 단어 생성 |


####  Decoder Layer 구성

1. **Masked Multi-Head Attention**
2. **Encoder-Decoder Attention (Cross-Attention)**
3. **Feed Forward Layer**

#####  Masked Multi-Head Attention
- 미래 단어를 참조하지 못하도록 **마스크(Mask)** 적용  
- 예: “I go to school” → “school”을 미리 보면 안 됨

#####  Encoder-Decoder Attention
- 인코더 출력(Context Vector)을 참조하여 현재 단어 생성  
- Query(Q): 디코더 상태  
- Key(K), Value(V): 인코더 출력  
- “school”을 생성할 때 “학교”에 집중 → 번역 품질 향상



## 4-3. 전체 모델 및 데이터 흐름



### 입력 준비 — 문장을 [의미 + 위치] 벡터로 변환
| 과정 | 설명 | 예시 |
|------|------|------|
| **Embedding** | 단어 → 의미 벡터 | `I`, `love`, `you` |
| **Positional Encoding** | 순서 벡터 추가 | 1번째 `I`, 2번째 `love`, 3번째 `you` |

 의미 + 위치 정보가 결합된 벡터가 인코더 입력으로 전달됨


###  인코더 — 문맥적 의미 파악
| 과정 | 설명 | 예시 |
|------|------|------|
| **Self-Attention** | 단어 간 관계 계산 | ‘love’ ↔ ‘I’, ‘you’ |
| **Softmax 가중치 계산** | Q·K 내적 → 확률 분포 | ‘love’가 ‘I’에 0.2, 자신에 0.7, ‘you’에 0.1 |
| **Weighted Sum** | 가중치×V 값의 합 | 새로운 `love` 벡터 생성 |
| **Multi-Head** | 다양한 관점 병렬 분석 | 주어-동사 / 동사-목적어 등 |

결과: 입력 문장의 **Context Vector 집합**



###  디코더 — 출력 문장 생성
| Step | 입력 | 주요 과정 | 출력 |
|------|------|------------|------|
| 1 | `<start>` | Masked Self-Attention + Cross-Attention → 인코더에서 ‘I’ 참조 | “나는” |
| 2 | `<start>`, “나는” | 새로운 Q 생성, 인코더 문맥 재참조 | “학교에” |
| 3 | `<start>`, “나는 학교에” | 계속 반복하며 예측 | “간다” |



**최종 결과:**  
트랜스포머는 인코더로 입력을 이해하고,  
디코더로 **자연스럽고 문맥적으로 맞는 문장**을 생성
